In [9]:
# This is a long document we can split up.
with open("/storage/data_4T_b/andreacutuli/PROVA/Scripts/result_document.md") as f:
    fac_simile = f.read()
print(fac_simile[:500])

## EcoTech Solutions S.r.l.

per GreenLife S.p.A. Via dell'Innovazione 42, 20121 Milano (MI), Italia

Soluzioni di Intelligenza Artificiale per l'Ottimizzazione del Processo di Riciclo Industriale

Implementazione di Sistemi AI per l'Analisi dei Materiali e l'Efficienza Energetica

## Proposta contrattuale

Versione: 1.3 Validità: 60 giorni

Destinatario: Marco Bianchi m.bianchi@greenlifespa.eu Laura Verdi l.verdi@greenlifespa.eu Andrea Rossi a.rossi@greenlifespa.eu

Redatta da: Dott.ssa Elena M


In [2]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface import HuggingFaceEmbeddings


# Usa un modello open-source per gli embeddings (gratuito, nessuna chiave API richiesta)
#embedding_model = HuggingFaceEmbeddings(model_name="Musixmatch/umberto-commoncrawl-cased-v1")
#embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

embedding_model = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large",
    #model_name="sentence-transformers/LaBSE"
    model_kwargs={"use_auth_token": "[REMOVED]"}
)


from sentence_transformers import SentenceTransformer
#model = SentenceTransformer("sentence-transformers/LaBSE")
model=embedding_model


# Inizializza il semantic chunker con quel modello
semantic_chunker = SemanticChunker(
    embeddings=embedding_model,
    breakpoint_threshold_type="gradient",
)


In [ ]:
'''semantic_chunker = SemanticChunker(
    embeddings=embedding_model,
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount=95,
    min_chunk_size=300,
    max_chunk_size=1000,
    buffer_size=50,
    embedding_batch_size=8
)
semantic_chunks = semantic_chunker.create_documents([fac_simile])'''

In [ ]:
semantic_chunks = semantic_chunker.create_documents([fac_simile])

In [ ]:
with open("semantic_chunks.md", "w", encoding="utf-8") as f:
    for i, doc in enumerate(semantic_chunks):
        f.write(f"### Chunk {i + 1}\n\n")
        f.write(doc.page_content.strip() + "\n\n")
print(f"File salvato come semantic_chunks.md")

print(f"Numero di chunk creati: {len(semantic_chunks)}")

In [ ]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface import HuggingFaceEmbeddings

# Lista modelli embedding da testare (se vuoi testarne più di uno)
embedding_models = [
    HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large", model_kwargs={"use_auth_token": "[REMOVED]"}),
    HuggingFaceEmbeddings(model_name="sentence-transformers/LaBSE"),
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"),
    HuggingFaceEmbeddings(model_name="nickprock/sentence-bert-base-italian-uncased"),
    HuggingFaceEmbeddings(model_name="nickprock/multi-sentence-BERTino", model_kwargs={"use_auth_token": "[REMOVED]"}),

]

# Lista di configurazioni per SemanticChunker da testare
chunker_configs = [
    # ● Solo “gradient” per transizioni semantiche pure, con contesto ampio
    {
        "breakpoint_threshold_type": "gradient",
        "buffer_size": 80,
        "embedding_batch_size": 16,
    },

    # ● Percentile (soglie alte → chunk più corposi)
    {
        "breakpoint_threshold_type": "percentile",
        "breakpoint_threshold_amount": 95,
        "buffer_size": 80,
        "embedding_batch_size": 16,
    },
    {
        "breakpoint_threshold_type": "percentile",
        "breakpoint_threshold_amount": 90,
        "buffer_size": 70,
        "embedding_batch_size": 16,
    },
    {
        "breakpoint_threshold_type": "percentile",
        "breakpoint_threshold_amount": 85,
        "buffer_size": 60,
        "embedding_batch_size": 12,
    },
    {
        "breakpoint_threshold_type": "percentile",
        "breakpoint_threshold_amount": 80,
        "buffer_size": 50,
        "embedding_batch_size": 8,
    },

    # ● Deviazione standard (multipli più alti → tagli più “accorti”)
    {
        "breakpoint_threshold_type": "standard_deviation",
        "breakpoint_threshold_amount": 1.0,
        "buffer_size": 80,
        "embedding_batch_size": 16,
    },
    {
        "breakpoint_threshold_type": "standard_deviation",
        "breakpoint_threshold_amount": 0.75,
        "buffer_size": 60,
        "embedding_batch_size": 12,
    },
    {
        "breakpoint_threshold_type": "standard_deviation",
        "breakpoint_threshold_amount": 0.5,
        "buffer_size": 40,
        "embedding_batch_size": 8,
    },

    # ● Interquartile range (IQR)
    {
        "breakpoint_threshold_type": "interquartile",
        "breakpoint_threshold_amount": 1.5,
        "buffer_size": 80,
        "embedding_batch_size": 16,
    },
    {
        "breakpoint_threshold_type": "interquartile",
        "breakpoint_threshold_amount": 1.0,
        "buffer_size": 60,
        "embedding_batch_size": 12,
    },
    {
        "breakpoint_threshold_type": "interquartile",
        "breakpoint_threshold_amount": 0.75,
        "buffer_size": 40,
        "embedding_batch_size": 8,
    },
]



In [13]:
# Parametri accettati da SemanticChunker (da adattare in base alla tua versione)
accepted_keys = {
    "breakpoint_threshold_type",
    "buffer_size",
    "chunk_size",
    "chunk_overlap",
    "breakpoint_threshold_amount",
    # aggiungi qui altri parametri validi se ce ne sono
}

counter = 45
for emb_model in embedding_models:
    for config in chunker_configs:
        print(f"Test numero {counter} con config: {config} e modello {emb_model.model_name}")
        
        # Filtra config per mantenere solo parametri accettati
        filtered_config = {k: v for k, v in config.items() if k in accepted_keys}

        semantic_chunker = SemanticChunker(
            embeddings=emb_model,
            **filtered_config
        )
        
        semantic_chunks = semantic_chunker.create_documents([fac_simile])
        
        filename = f"semantic_chunks_{counter}.md"
        with open(filename, "w", encoding="utf-8") as f:
            for i, doc in enumerate(semantic_chunks):
                f.write(f"### Chunk {i + 1}\n\n")
                f.write(doc.page_content.strip() + "\n\n")
        
        print(f"File salvato come {filename}")
        print(f"Numero di chunk creati: {len(semantic_chunks)}\n")
        
        counter += 1


Test numero 45 con config: {'breakpoint_threshold_type': 'gradient', 'buffer_size': 80, 'embedding_batch_size': 16} e modello nickprock/multi-sentence-BERTino
File salvato come semantic_chunks_45.md
Numero di chunk creati: 13

Test numero 46 con config: {'breakpoint_threshold_type': 'percentile', 'breakpoint_threshold_amount': 95, 'buffer_size': 80, 'embedding_batch_size': 16} e modello nickprock/multi-sentence-BERTino
File salvato come semantic_chunks_46.md
Numero di chunk creati: 13

Test numero 47 con config: {'breakpoint_threshold_type': 'percentile', 'breakpoint_threshold_amount': 90, 'buffer_size': 70, 'embedding_batch_size': 16} e modello nickprock/multi-sentence-BERTino
File salvato come semantic_chunks_47.md
Numero di chunk creati: 24

Test numero 48 con config: {'breakpoint_threshold_type': 'percentile', 'breakpoint_threshold_amount': 85, 'buffer_size': 60, 'embedding_batch_size': 12} e modello nickprock/multi-sentence-BERTino
File salvato come semantic_chunks_48.md
Numero di

In [ ]:
#CONFIGUTRAZIONE CONSIGLIATA DA CHAT GPT
semantic_chunker = SemanticChunker(
    embeddings=embedding_model,
    breakpoint_threshold_type="gradient",  # o "percentile" se vuoi sperimentare con soglie fisse
    buffer_size=80,  # aumenta il contesto per rendere i chunk più coerenti
    embedding_batch_size=16  # sfrutta la potenza del tuo computer
)